In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json
from flatten_json import flatten

In [2]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-2a11ccd9-3521-4259-8999-2bc2c8dab556"

In [3]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://na1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/MASTER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [4]:
sumids[0:5]

['ne2x4D9Iw2seal8TL0XQCTNmWleBZKlfS93s33128QinNfM',
 '7yo0fGP6cYQRlxdlunvQwFzK5HXsMi2XzVkAuuMbQQRB6hY',
 'MsE5_TjrZ3xQMNv6RnY779vGUIPEZOUu2duYTrL1Nwche2Gv',
 'FbUPULPW4KvVhtxmbbAVHiVufeu14nEtx0FXK3wnbkPLu0I',
 'qyPpX6oCAQiEnEFAU_RpGJOPvhz8sBLrI5iFwWCloibxenc']

In [5]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://na1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [6]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
    assert matlreq.status_code == 200
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [7]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = set(gamids)

print(len(gamids))

20500
17771


In [8]:
with open('chall_gam_id_list.txt', 'r') as f:
    challgames = f.read().splitlines()

In [9]:
challgames =set([int(e) for e in challgames])

In [11]:
with open('gm_gam_id_list.txt', 'r') as f:
    gmgames = f.read().splitlines()

In [12]:
gmgames =set([int(e) for e in gmgames])

In [22]:
gamids = gamids - (gamids & (gmgames | challgames))

In [ ]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids:
    gamreq = requests.get("https://na1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://na1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            break
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [55]:
gamreq.status_code

200

In [56]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(10724, 1550, 133)

In [57]:
# create df from matches

df = pd.DataFrame(matchlist)

In [58]:
df1 = df[df['gameMode'] == 'CLASSIC']

In [59]:
df1 = df1.drop(["platformId","seasonId",'queueId','mapId'],axis=1)

In [60]:
df1

,gameId,gameCreation,gameDuration,gameVersion,gameMode,gameType,teams_0_teamId,teams_0_win,teams_0_firstBlood,teams_0_firstTower,...,participants_2_timeline_damageTakenDiffPerMinDeltas_30-end,participants_4_timeline_csDiffPerMinDeltas_30-end,participants_4_timeline_xpDiffPerMinDeltas_30-end,participants_4_timeline_damageTakenDiffPerMinDeltas_30-end,participants_5_timeline_csDiffPerMinDeltas_30-end,participants_5_timeline_xpDiffPerMinDeltas_30-end,participants_5_timeline_damageTakenDiffPerMinDeltas_30-end,participants_6_timeline_csDiffPerMinDeltas_30-end,participants_6_timeline_xpDiffPerMinDeltas_30-end,participants_6_timeline_damageTakenDiffPerMinDeltas_30-end
0,3500081156,1595189917997,1430,10.14.327.8505,CLASSIC,MATCHED_GAME,100,Win,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3467280399,1592606401103,1468,10.12.325.6222,CLASSIC,MATCHED_GAME,100,Fail,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3468886033,1592718491369,1085,10.12.325.6222,CLASSIC,MATCHED_GAME,100,Fail,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3473539090,1593113278238,1851,10.13.325.7485,CLASSIC,MATCHED_GAME,100,Fail,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3500867605,1595221306966,2226,10.14.327.8505,CLASSIC,MATCHED_GAME,100,Fail,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10718,3477536751,1593430520991,634,10.13.326.4870,CLASSIC,MATCHED_GAME,100,Fail,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10719,3486318579,1594159625393,1264,10.13.326.4870,CLASSIC,MATCHED_GAME,100,Win,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10721,3477929977,1593467680057,918,10.13.326.4870,CLASSIC,MATCHED_GAME,100,Fail,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10722,3492577275,1594606823570,1571,10.14.327.8505,CLASSIC,MATCHED_GAME,100,Win,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
len(set(list(df['gameId'])))

10724

In [63]:
compression_opts = dict(method='zip',
                        archive_name='master.csv')  

In [64]:
df.to_csv('master.zip', index=False,
          compression=compression_opts)